<a href="https://colab.research.google.com/github/maksim267/Credit_Scoring/blob/main/%D0%9A%D1%80%D0%B5%D0%B4%D0%B8%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%BA%D0%BE%D1%80%D0%B8%D0%BD%D0%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Кредитный скоринг

# Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive/')#/content/gdrive/MyDrive/journey-springfield.zip
# !unzip -q /content/gdrive/MyDrive/findata-creditscoring.zip

In [ ]:
data = pd.read_csv('./credit_train.csv')

In [ ]:
data

# Работа над пропущенными значениями

In [ ]:
data["Customer ID"].value_counts()# pfvtnbv

In [ ]:
data.loc[data["Customer ID"].map(data["Customer ID"].value_counts()) > 1, "Number of Occurrences"] = 1
data.loc[data["Customer ID"].map(data["Customer ID"].value_counts()) == 1, "Number of Occurrences"] = 0
data = data.drop(columns=["Customer ID","Loan ID"])

NameError: name 'data' is not defined

In [ ]:
data.isna().sum(axis=0)

In [ ]:
data["Credit Score"].value_counts().sort_index()

In [ ]:
data[["Annual Income","Credit Score"]].sort_values(by="Annual Income", ascending=True)

NameError: name 'data' is not defined

In [ ]:
data["Annual Income"] =data["Annual Income"].fillna(data['Annual Income'].median())
data["Credit Score"]=data["Credit Score"].fillna(data['Credit Score'].median())

In [ ]:
data["Years in current job"].value_counts()

In [ ]:
data["Years in current job"]=data["Years in current job"].fillna(-1)

In [ ]:
data["Months since last delinquent"].value_counts().sort_index()

In [ ]:
# data = data.drop(columns=["Months since last delinquent"])
data["Months since last delinquent"]=data["Months since last delinquent"].fillna(-1)

NameError: name 'data' is not defined

In [ ]:
data = data.drop(index=data[pd.isna(data["Maximum Open Credit"])].index).reset_index(drop=True)

In [ ]:
data["Bankruptcies"].value_counts().sort_index()

NameError: name 'data' is not defined

In [ ]:
# data = data.drop(columns=["Bankruptcies"])
data["Bankruptcies"]=data["Bankruptcies"].fillna(0)

In [ ]:
data = data.drop(index=data[pd.isna(data["Tax Liens"])].index).reset_index(drop=True)

In [ ]:
data.isna().sum()

# Ручное преобразование некоторых категориальных признаков OneHotEncoding над остальными

In [ ]:
data

In [ ]:
data.dtypes

In [ ]:
k=0
# посмотрим на то какие значения принимают категориальные признаки
for col in data.columns[data.dtypes=='object']:
  k+=1
  plt.figure(figsize=(30,30))
  val = data[col].value_counts()
  plt.subplot(8, 2, k)
  plt.pie(val.values, labels=val.index, autopct='%1.1f%%')
  plt.title(col)

In [ ]:
data.loc[data['Loan Status']=='Fully Paid', 'Loan Status']=1
data.loc[data['Loan Status']=='Charged Off', 'Loan Status']=0
data["Loan Status"]=data['Loan Status'].astype(int)

data.loc[data['Term']=='Long Term', 'Term']=1
data.loc[data['Term']=='Short Term', 'Term']=0
data["Term"]=data['Term'].astype(int)

conversion_dict = {
    "8 years": 8,
    "10+ years": 10,
    "3 years": 3,
    "5 years": 5,
    "7 years": 7,
    -1: -1,
    "1 year": 1,
    "2 years": 2,
    "4 years": 4,
    "6 years": 6,
    "9 years": 9,
    "< 1 year": 0
}
data["Years in current job"] = data["Years in current job"].map(conversion_dict)
data["Years in current job"]=data['Years in current job'].astype(int)

In [ ]:
data

In [ ]:
data["Purpose"].value_counts()

In [ ]:
data["Purpose"] = data["Purpose"].str.lower()

NameError: name 'data' is not defined

In [ ]:
data.dtypes

In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline

real_columns = np.hstack([np.array(data.columns[data.dtypes=='int64']),np.array(data.columns[data.dtypes=='float64']) ])# выделение вещественных признаков
real_columns=real_columns[real_columns!='Loan Status']
real_data = data[real_columns]
transformSS = StandardScaler().fit(real_data)
real_dataSS=transformSS.transform(real_data)
# real_dataSS=real_data

cat_columns=data.columns[data.dtypes=='object']# выделение категориальных признаков
cat_data = data[cat_columns]

encoder = OneHotEncoder(sparse_output=False)
one_hot_data = encoder.fit(cat_data)
one_hot_data = pd.DataFrame(one_hot_data.transform(cat_data), index=cat_data.index, columns=encoder.get_feature_names_out())
new_data = pd.concat([pd.DataFrame(real_dataSS, columns=real_columns), one_hot_data], axis='columns')

NameError: name 'np' is not defined

# Анализ данных и работа над выбрасами

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
new_data[real_columns]

In [ ]:
plt.figure(figsize=(40,15))
sns.boxplot(data=new_data[real_columns])
plt.show()

In [ ]:
max(new_data["Maximum Open Credit"])

In [ ]:
  new_data['Maximum Open Credit'].mean()

In [ ]:
new_data.loc[new_data["Maximum Open Credit"]<=new_data['Maximum Open Credit'].mean()*10]

In [ ]:
new_data = new_data.drop(index=new_data[new_data["Maximum Open Credit"]>new_data['Maximum Open Credit'].mean()*5].index)

In [ ]:
new_data

In [ ]:
plt.figure(figsize=(40,15))
sns.boxplot(data=new_data[real_columns])
plt.show()

In [ ]:
max(new_data["Annual Income"])

In [ ]:
sns.histplot(data=new_data, x="Annual Income", bins=100, color='blue')  # Добавляем KDE (плавную линию)
plt.xlim(0, 10000000)
plt.xlabel("Диапазоны значений")
plt.ylabel("Количество значений")
plt.show()

In [ ]:
new_data = new_data.drop(index=new_data[new_data["Annual Income"]>new_data["Annual Income"].mean()*4].index)

In [ ]:
plt.figure(figsize=(40,15))
sns.boxplot(data=new_data[real_columns])
plt.show()

In [ ]:
new_data

In [ ]:
new_data = new_data.drop(index=new_data[(new_data["Current Loan Amount"]==max(new_data["Current Loan Amount"]))].index)
new_data = new_data.drop(index=new_data[(new_data["Current Credit Balance"]==new_data["Current Credit Balance"].mean()*5)].index)

In [ ]:
data

In [ ]:
plt.figure(figsize=(40,15))
sns.boxplot(new_data[real_columns])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for column in real_columns:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=new_data[column])
    plt.title(f'Boxplot для {column}')
    plt.xlabel(column)
    plt.show()


NameError: name 'real_columns' is not defined